In [1]:
# importing the required libraries
import numpy as np
import pandas as pd

In [2]:
# opening the file
with open('C:/Users/babli/OneDrive/Desktop/MAPUP/access_log', 'r', encoding='utf-8', errors='ignore') as file:
    lines = file.readlines()


In [3]:
# printing few lines of the data
for line in lines[:5]:
    print(line.strip())


local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150
local - - [24/Oct/1994:13:41:41 -0600] "GET 1.gif HTTP/1.0" 200 1210
local - - [24/Oct/1994:13:43:13 -0600] "GET index.html HTTP/1.0" 200 3185
local - - [24/Oct/1994:13:43:14 -0600] "GET 2.gif HTTP/1.0" 200 2555
local - - [24/Oct/1994:13:43:15 -0600] "GET 3.gif HTTP/1.0" 200 36403


In [4]:
# importing re for regex
import re

data = []

# Regular expression pattern for standard Common Log Format (CLF) 
# local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150

pattern = r'(\S+) (\S+) (\S+) \[(.*?)\] "(.*?)" (\d{3}) (\d+)'

for line in lines:
    match = re.match(pattern, line)
    if match:
        host = match.group(1)
        rfc931 = match.group(2)
        authuser = match.group(3)
        timestamp = match.group(4)
        filename = match.group(5)  # includes method, file, protocol
        http_code = match.group(6)
        bytes_sent = match.group(7)

        data.append([host, rfc931, authuser, timestamp, filename, http_code, bytes_sent])

# Create a DataFrame
df = pd.DataFrame(data, columns=['host', 'rfc931', 'authuser', 'timestamp', 'filename', 'http_code', 'bytes'])

# Display the first few rows
df.head()


,host,rfc931,authuser,timestamp,filename,http_code,bytes
0,local,-,-,24/Oct/1994:13:41:41 -0600,GET index.html HTTP/1.0,200,150
1,local,-,-,24/Oct/1994:13:41:41 -0600,GET 1.gif HTTP/1.0,200,1210
2,local,-,-,24/Oct/1994:13:43:13 -0600,GET index.html HTTP/1.0,200,3185
3,local,-,-,24/Oct/1994:13:43:14 -0600,GET 2.gif HTTP/1.0,200,2555
4,local,-,-,24/Oct/1994:13:43:15 -0600,GET 3.gif HTTP/1.0,200,36403


In [5]:
df.shape

(666804, 7)

In [6]:
# checking unique values for all the columns
for col in df.columns:
    print(f"column name : {col},\n unique records: ", df[col].unique())
    print('=' * 50,'\n')

column name : host,
 unique records:  ['local' 'remote']

column name : rfc931,
 unique records:  ['-']

column name : authuser,
 unique records:  ['-' 'Villain']

column name : timestamp,
 unique records:  ['24/Oct/1994:13:41:41 -0600' '24/Oct/1994:13:43:13 -0600'
 '24/Oct/1994:13:43:14 -0600' ... '11/Oct/1995:14:11:39 -0600'
 '11/Oct/1995:14:11:40 -0600' '11/Oct/1995:14:14:17 -0600']

column name : filename,
 unique records:  ['GET index.html HTTP/1.0' 'GET 1.gif HTTP/1.0' 'GET 2.gif HTTP/1.0' ...
 'GET 12178.gif HTTP/1.0' 'GET 11624.html HTTP/1.0'
 'GET 12179.gif HTTP/1.0']

column name : http_code,
 unique records:  ['200' '304' '302' '500' '403']

column name : bytes,
 unique records:  ['150' '1210' '3185' ... '144236' '116823' '11027']



In [7]:
df["timestamp"].unique()

array(['24/Oct/1994:13:41:41 -0600', '24/Oct/1994:13:43:13 -0600',
       '24/Oct/1994:13:43:14 -0600', ..., '11/Oct/1995:14:11:39 -0600',
       '11/Oct/1995:14:11:40 -0600', '11/Oct/1995:14:14:17 -0600'],
      dtype=object)

In [8]:
from datetime import datetime

# Removing timezone while parsing
df['timestamp'] = df['timestamp'].apply(lambda x: datetime.strptime(x.split(' ')[0], '%d/%b/%Y:%H:%M:%S'))
df['timestamp']

0        1994-10-24 13:41:41
1        1994-10-24 13:41:41
2        1994-10-24 13:43:13
3        1994-10-24 13:43:14
4        1994-10-24 13:43:15
                 ...        
666799   1995-10-11 14:10:59
666800   1995-10-11 14:11:38
666801   1995-10-11 14:11:39
666802   1995-10-11 14:11:40
666803   1995-10-11 14:14:17
Name: timestamp, Length: 666804, dtype: datetime64[ns]

In [9]:
df["timestamp"].unique()

<DatetimeArray>
['1994-10-24 13:41:41', '1994-10-24 13:43:13', '1994-10-24 13:43:14',
 '1994-10-24 13:43:15', '1994-10-24 13:43:17', '1994-10-24 13:46:45',
 '1994-10-24 13:46:47', '1994-10-24 13:46:50', '1994-10-24 13:47:19',
 '1994-10-24 13:47:41',
 ...
 '1995-10-11 14:07:28', '1995-10-11 14:07:30', '1995-10-11 14:09:41',
 '1995-10-11 14:10:57', '1995-10-11 14:10:58', '1995-10-11 14:10:59',
 '1995-10-11 14:11:38', '1995-10-11 14:11:39', '1995-10-11 14:11:40',
 '1995-10-11 14:14:17']
Length: 561746, dtype: datetime64[ns]

In [10]:
df["filename"].unique()

array(['GET index.html HTTP/1.0', 'GET 1.gif HTTP/1.0',
       'GET 2.gif HTTP/1.0', ..., 'GET 12178.gif HTTP/1.0',
       'GET 11624.html HTTP/1.0', 'GET 12179.gif HTTP/1.0'], dtype=object)

In [11]:
#checking the quantum of null values corresponding to every columns 
df.isnull().sum()

host         0
rfc931       0
authuser     0
timestamp    0
filename     0
http_code    0
bytes        0
dtype: int64

In [12]:
# replacing "-" with unavailable for column "rfc931"
df['rfc931'] = df['rfc931'].replace("-", "unavailable").astype(str)

In [13]:
# replacing "-" with not authenticated for column "authuser"
df['authuser'] = df['authuser'].replace("-", "not authenticated").astype(str)

In [14]:
# checking the uniqueness of column "rfc931"
df['rfc931'].unique()

array(['unavailable'], dtype=object)

In [15]:
# checking the uniqueness of column "authuser"
df['authuser'].unique()

array(['not authenticated', 'Villain'], dtype=object)

In [16]:
# final check of unique records of columns
for col in df.columns:
    print(f"column name : {col},\n unique records: ", df[col].unique())
    print('=' * 50,'\n')

column name : host,
 unique records:  ['local' 'remote']

column name : rfc931,
 unique records:  ['unavailable']

column name : authuser,
 unique records:  ['not authenticated' 'Villain']

column name : timestamp,
 unique records:  <DatetimeArray>
['1994-10-24 13:41:41', '1994-10-24 13:43:13', '1994-10-24 13:43:14',
 '1994-10-24 13:43:15', '1994-10-24 13:43:17', '1994-10-24 13:46:45',
 '1994-10-24 13:46:47', '1994-10-24 13:46:50', '1994-10-24 13:47:19',
 '1994-10-24 13:47:41',
 ...
 '1995-10-11 14:07:28', '1995-10-11 14:07:30', '1995-10-11 14:09:41',
 '1995-10-11 14:10:57', '1995-10-11 14:10:58', '1995-10-11 14:10:59',
 '1995-10-11 14:11:38', '1995-10-11 14:11:39', '1995-10-11 14:11:40',
 '1995-10-11 14:14:17']
Length: 561746, dtype: datetime64[ns]

column name : filename,
 unique records:  ['GET index.html HTTP/1.0' 'GET 1.gif HTTP/1.0' 'GET 2.gif HTTP/1.0' ...
 'GET 12178.gif HTTP/1.0' 'GET 11624.html HTTP/1.0'
 'GET 12179.gif HTTP/1.0']

column name : http_code,
 unique records:  [

In [17]:
# Step 1: Getting path from request
df['request_path'] = df['filename'].apply(
    lambda x: x.split()[1] if isinstance(x, str) and len(x.split()) > 1 else 'none'
)

# Step 2: Extracting extension from path
df['file_extension'] = df['request_path'].str.extract(r'\.([a-zA-Z0-9]+)$')


In [18]:
# counting unique value of column file extension
df['file_extension'].value_counts()

file_extension
html       296858
gif        290721
jpg         41641
xbm         15905
ps           5461
            ...  
test            1
nanda           1
human           1
cheryl          1
meeting         1
Name: count, Length: 132, dtype: int64

In [19]:
# converting the data type appropriately

df['http_code'] = df['http_code'].astype(int)
df['bytes'] = df['bytes'].astype(int)
df['file_extension'] = df['file_extension'].fillna('unknown').astype(str)
df['filename'] = df['filename'].astype(str)

In [20]:
# checking the overall information of all columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666804 entries, 0 to 666803
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   host            666804 non-null  object        
 1   rfc931          666804 non-null  object        
 2   authuser        666804 non-null  object        
 3   timestamp       666804 non-null  datetime64[ns]
 4   filename        666804 non-null  object        
 5   http_code       666804 non-null  int32         
 6   bytes           666804 non-null  int32         
 7   request_path    666804 non-null  object        
 8   file_extension  666804 non-null  object        
dtypes: datetime64[ns](1), int32(2), object(6)
memory usage: 40.7+ MB


In [21]:
#checking null values
df.isnull().sum()

host              0
rfc931            0
authuser          0
timestamp         0
filename          0
http_code         0
bytes             0
request_path      0
file_extension    0
dtype: int64

In [22]:
df.head()

,host,rfc931,authuser,timestamp,filename,http_code,bytes,request_path,file_extension
0,local,unavailable,not authenticated,1994-10-24 13:41:41,GET index.html HTTP/1.0,200,150,index.html,html
1,local,unavailable,not authenticated,1994-10-24 13:41:41,GET 1.gif HTTP/1.0,200,1210,1.gif,gif
2,local,unavailable,not authenticated,1994-10-24 13:43:13,GET index.html HTTP/1.0,200,3185,index.html,html
3,local,unavailable,not authenticated,1994-10-24 13:43:14,GET 2.gif HTTP/1.0,200,2555,2.gif,gif
4,local,unavailable,not authenticated,1994-10-24 13:43:15,GET 3.gif HTTP/1.0,200,36403,3.gif,gif


# Part 2: Analysis Questions (5 marks each)

### Q1: Count of total log records

In [23]:
total_log_records = len(df)
print(total_log_records)


666804


## Q2: Count of unique hosts

In [24]:
df['host'].nunique()

2

## Q3: Date-wise unique filename counts 

In [25]:
# Group by date and count unique filenames per date
df['date_format'] = df['timestamp'].dt.strftime('%d-%b-%Y')
unique_filenames_per_date =dict(df.groupby('date_format')['filename'].nunique().sort_values(ascending = False))
unique_filenames_per_date

{'15-Mar-1995': 900,
 '04-Oct-1995': 879,
 '12-Apr-1995': 847,
 '05-Apr-1995': 844,
 '06-Oct-1995': 835,
 '26-Sep-1995': 831,
 '02-Oct-1995': 828,
 '29-Mar-1995': 827,
 '03-Oct-1995': 825,
 '10-Oct-1995': 823,
 '13-Mar-1995': 818,
 '28-Sep-1995': 818,
 '05-Oct-1995': 814,
 '30-Mar-1995': 813,
 '29-Sep-1995': 805,
 '27-Sep-1995': 798,
 '11-Apr-1995': 795,
 '28-Mar-1995': 794,
 '20-Sep-1995': 793,
 '27-Mar-1995': 789,
 '04-Apr-1995': 775,
 '10-May-1995': 770,
 '15-Feb-1995': 769,
 '02-Aug-1995': 766,
 '14-Mar-1995': 757,
 '21-Sep-1995': 749,
 '09-May-1995': 749,
 '03-Apr-1995': 747,
 '21-Mar-1995': 744,
 '09-Oct-1995': 732,
 '08-Sep-1995': 731,
 '10-Apr-1995': 728,
 '13-Sep-1995': 728,
 '07-Mar-1995': 724,
 '09-Mar-1995': 722,
 '23-Mar-1995': 721,
 '07-Apr-1995': 715,
 '25-Sep-1995': 713,
 '11-Oct-1995': 709,
 '19-Sep-1995': 707,
 '07-May-1995': 707,
 '31-Mar-1995': 707,
 '11-Sep-1995': 691,
 '10-Feb-1995': 690,
 '14-Sep-1995': 686,
 '14-Feb-1995': 684,
 '02-May-1995': 681,
 '19-Apr-1995

## Q4: Number of 404 response codes

In [26]:
count_404 = df[df['http_code'] == 404].shape[0]
print(count_404)

0


In [27]:
# verifying data of http_code=404
df[df["http_code"]==404]

,host,rfc931,authuser,timestamp,filename,http_code,bytes,request_path,file_extension,date_format


## Q5: Top 15 filenames with 404 responses

In [28]:
# If http_code is int, comparing with integer 404 instead of string '404'
df_404 = df[df['http_code'] == 404]

top_15_404 = df_404['filename'].value_counts().head(15)

top_15_404_list = list(top_15_404.items())

print(top_15_404_list)

[]


## Q6: Top 15 file extensions with 404 responses

In [29]:
# If http_code is int, comparing with integer 404 instead of string '404'
df_404 = df[df['http_code'] == 404]

top_15_404 = df_404['file_extension'].value_counts().head(15)

top_15_404_list = list(top_15_404.items())

print(top_15_404_list)


[]


In [30]:
df.dtypes

host                      object
rfc931                    object
authuser                  object
timestamp         datetime64[ns]
filename                  object
http_code                  int32
bytes                      int32
request_path              object
file_extension            object
date_format               object
dtype: object

## Q7: Total bandwidth transferred per day for July 1995


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666804 entries, 0 to 666803
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   host            666804 non-null  object        
 1   rfc931          666804 non-null  object        
 2   authuser        666804 non-null  object        
 3   timestamp       666804 non-null  datetime64[ns]
 4   filename        666804 non-null  object        
 5   http_code       666804 non-null  int32         
 6   bytes           666804 non-null  int32         
 7   request_path    666804 non-null  object        
 8   file_extension  666804 non-null  object        
 9   date_format     666804 non-null  object        
dtypes: datetime64[ns](1), int32(2), object(7)
memory usage: 45.8+ MB


In [32]:
df[df['bytes'] == "-"]

,host,rfc931,authuser,timestamp,filename,http_code,bytes,request_path,file_extension,date_format


In [33]:
# Converting timestamp column to datetime (if not already)
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Filtering for July 1995 only
july_1995 = df[(df['timestamp'].dt.year == 1995) & (df['timestamp'].dt.month == 7)].copy()

# Converting bytes to numeric, coercing errors (e.g. '-' to NaN)
july_1995['bytes'] = pd.to_numeric(july_1995['bytes'], errors='coerce')

# Dropping rows with missing bytes
july_1995 = july_1995.dropna(subset=['bytes'])

# Grouping by date and sum bytes
july_1995['date_str'] = july_1995['timestamp'].dt.strftime('%d-%b-%Y')
bandwidth_per_day = july_1995.groupby('date_str')['bytes'].sum().astype(int)

# Converting to dict
bandwidth_dict = bandwidth_per_day.to_dict()

bandwidth_dict


{'01-Jul-1995': 11349799,
 '02-Jul-1995': 8656918,
 '03-Jul-1995': 13596612,
 '04-Jul-1995': 26573988,
 '05-Jul-1995': 19541225,
 '06-Jul-1995': 19755015,
 '07-Jul-1995': 9427822,
 '08-Jul-1995': 5403491,
 '09-Jul-1995': 4660556,
 '10-Jul-1995': 14917754,
 '11-Jul-1995': 22507207,
 '12-Jul-1995': 17367065,
 '13-Jul-1995': 15989234,
 '14-Jul-1995': 19186430,
 '15-Jul-1995': 15773233,
 '16-Jul-1995': 9016378,
 '17-Jul-1995': 19601338,
 '18-Jul-1995': 17099761,
 '19-Jul-1995': 17851725,
 '20-Jul-1995': 20752623,
 '21-Jul-1995': 25491617,
 '22-Jul-1995': 8136259,
 '23-Jul-1995': 9593870,
 '24-Jul-1995': 22308265,
 '25-Jul-1995': 24561635,
 '26-Jul-1995': 24995540,
 '27-Jul-1995': 25969995,
 '28-Jul-1995': 36460693,
 '29-Jul-1995': 11700624,
 '30-Jul-1995': 23189598,
 '31-Jul-1995': 30730715}

## Q8: Hourly request distribution

In [34]:
# Ensuring 'timestamp' is datetime type
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Extracting hour from timestamp
df['hour'] = df['timestamp'].dt.hour

# Counting requests per hour
hourly_counts = df['hour'].value_counts().sort_index()

# Converting to dict
hourly_distribution = hourly_counts.to_dict()

hourly_distribution


{0: 17206,
 1: 13196,
 2: 11733,
 3: 10027,
 4: 9369,
 5: 10143,
 6: 12280,
 7: 15550,
 8: 24664,
 9: 31115,
 10: 39713,
 11: 43766,
 12: 43085,
 13: 47338,
 14: 50087,
 15: 46337,
 16: 46849,
 17: 41378,
 18: 30435,
 19: 28219,
 20: 27169,
 21: 25109,
 22: 21873,
 23: 20163}

## Q9: Top 10 most requested filenames

In [35]:
top_10_files = df['filename'].value_counts().sort_values(ascending= False).head(10)
final = list(top_10_files.items())
final

[('GET index.html HTTP/1.0', 103825),
 ('GET 3.gif HTTP/1.0', 24001),
 ('GET 2.gif HTTP/1.0', 23590),
 ('GET 4.gif HTTP/1.0', 8013),
 ('GET 244.gif HTTP/1.0', 5143),
 ('GET 5.html HTTP/1.0', 5004),
 ('GET 4097.gif HTTP/1.0', 4865),
 ('GET 8870.jpg HTTP/1.0', 4488),
 ('GET 6733.gif HTTP/1.0', 4278),
 ('GET 8472.gif HTTP/1.0', 3843)]

## Q10: HTTP response code distribution

In [36]:
code = df['http_code'].value_counts() 
count_http = dict(code.items())
count_http

{200: 568348, 304: 97792, 302: 640, 500: 22, 403: 2}